# Test bug on RAGxplorer on query
Without brackets around the query in the openai client with chroma, embeddings are created for each word, not each string.

In [7]:
import os
from ragxplorer import RAGxplorer

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)
import json
import chromadb
import pandas as pd

In [8]:
query="What are examples of lubricants which should be avoided for space mechanism applications?"

OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
embedding_model='text-embedding-ada-002'

data_path='../data/AMS/'
pdf='AMS_2022.pdf'

index_path='../db/chromadb/'
index_name='chromadb-openai-ams'

viz_data='../data/AMS/ams_data-400-0-50.json'

In [9]:
with open(viz_data, 'r') as f:
    data = json.load(f)

index_name=data['visualization_index_name']
umap_params=data['umap_params']
viz_data=pd.read_json(data['viz_data'], orient='split')

/var/folders/jc/np72ktks0pg2f0g87j_0c0m80000gn/T/ipykernel_23823/2399012378.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  viz_data=pd.read_json(data['viz_data'], orient='split')


In [10]:
rx_client = RAGxplorer(embedding_model=embedding_model)
chroma_client = chromadb.PersistentClient(path='../db'+'/chromadb/')
collection=chroma_client.get_collection(name=index_name,embedding_function=rx_client._chosen_embedding_model)

Embedding model chosen: text-embedding-ada-002


In [11]:
rx_client._query.original_query = query

In [12]:
rx_client.load_chroma(collection,
                     umap_params=umap_params,
                     initialize_projector=True)

Setting up umap projector


/Users/danmueller/Repositories/Github/me/aerospace_chatbot/.venv/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [13]:
# All of these do the same thing, tested

query_embedding_chroma=rx_client._chosen_embedding_model([rx_client._query.original_query])
# query_embedding=rx_client._vectordb._embedding_function(rx_client._query.original_query)
# query_embedding=embedding_function(query)

In [14]:
len(rx_client._query.original_query)

'What are examples of lubricants which should be avoided for space mechanism applications?'

In [ ]:
fig = rx_client.visualize_query(query,
                                import_projection_data=viz_data,
                                verbose=True)

In [20]:
from openai import OpenAI
client = OpenAI()

# Done correctly, see here: https://platform.openai.com/docs/guides/embeddings/use-cases
openai_embedding=client.embeddings.create(input = [query], model=embedding_model).data[0].embedding